In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import Data for V1.1 Feature Selection

In [80]:
X_train= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/x_Train.csv')
X_test= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/x_Test.csv')
y_train= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/y_Train.csv')
y_test= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/y_Test.csv')
finalDF = pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.4/FinalTest.csv')

In [81]:
columnList1 = []
for i in finalDF.columns:
    columnList1.append(i)
    for i in X_train.columns:
        columnList1.append(i)
        for i in X_test.columns:
            columnList1.append(i)
columnSet = set(columnList1)


{'Alley_None',
 'Alley_Pave',
 'AvgConQual',
 'BldgType_2fmCon',
 'BldgType_Duplex',
 'BldgType_Twnhs',
 'BldgType_TwnhsE',
 'BsmtExposure_Gd',
 'BsmtExposure_Mn',
 'BsmtExposure_No',
 'BsmtExposure_None',
 'BsmtFinish',
 'BsmtPerFin',
 'CentralAir_Y',
 'Condition1_Feedr',
 'Condition1_Norm',
 'Condition1_PosA',
 'Condition1_PosN',
 'Condition1_RRAe',
 'Condition1_RRAn',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Condition2_Feedr',
 'Condition2_Norm',
 'Condition2_PosA',
 'Condition2_PosN',
 'Condition2_RRNn',
 'Electrical_FuseF',
 'Electrical_FuseP',
 'Electrical_SBrkr',
 'Exterior1st_AsphShn',
 'Exterior1st_BrkComm',
 'Exterior1st_BrkFace',
 'Exterior1st_CemntBd',
 'Exterior1st_HdBoard',
 'Exterior1st_MetalSd',
 'Exterior1st_None',
 'Exterior1st_Plywood',
 'Exterior1st_Stone',
 'Exterior1st_Stucco',
 'Exterior1st_VinylSd',
 'Exterior1st_Wd Sdng',
 'Exterior1st_WdShing',
 'Exterior2nd_AsphShn',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_BrkFace',
 'Exterior2nd_CBlock',
 'Exterior2nd_CmentBd

In [148]:
columnList2 = list(columnSet)
len(columnList2)

196

In [152]:
addList = []
for i in columnList2:
    if i not in X_train.columns:
        addList.append(i)
X_train[addList] = 0
addList = []
for i in columnList2:
    if i not in finalDF.columns:
        addList.append(i)
finalDF[addList] = 0
addList = []
for i in columnList2:
    if i not in X_test.columns:
        addList.append(i)
X_test[addList] = 0

In [153]:
len(X_test.columns)

196

# Fit an ElasticNet Model With GridSearch

In [84]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [154]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
parametersGrid = {"max_iter": [1, 5, 10],
                "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                "l1_ratio": np.arange(0.0, 1.0, 0.1)}
eNet = ElasticNet(normalize=True)
grid = GridSearchCV(eNet, parametersGrid, scoring='r2', cv=10)
grid.fit(X_train, y_train)
pred = grid.predict(X_test)
    

In [155]:
PredGrid = pd.concat([pd.Series(pred), y_test], axis = 1)
PredGrid.columns = ['Predicted', 'Actual']
PredGrid

,Predicted,Actual
0,194028.747739,173000
1,180063.554362,135000
2,262257.165281,267000
3,149214.195912,161000
4,193498.706182,136000
...,...,...
428,109061.925443,135750
429,196448.084812,192500
430,157902.336025,146500
431,114772.204684,118500


## Lets score ourselves

In [156]:
PredGrid['Difference']=(PredGrid['Predicted'] - PredGrid['Actual'])**2
PredGrid.Difference.sum()


523308435775.5463

In [157]:
PredGrid.Difference.mean()**.5


34764.41451833538

In [158]:
grid.score(X_test, y_test)

0.8204553636062574

In [159]:
grid.get_params()

{'cv': 10,
 'error_score': nan,
 'estimator__alpha': 1.0,
 'estimator__copy_X': True,
 'estimator__fit_intercept': True,
 'estimator__l1_ratio': 0.5,
 'estimator__max_iter': 1000,
 'estimator__normalize': True,
 'estimator__positive': False,
 'estimator__precompute': False,
 'estimator__random_state': None,
 'estimator__selection': 'cyclic',
 'estimator__tol': 0.0001,
 'estimator__warm_start': False,
 'estimator': ElasticNet(normalize=True),
 'n_jobs': None,
 'param_grid': {'max_iter': [1, 5, 10],
  'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
  'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': 'r2',
 'verbose': 0}

## Lets try a Lasso Model

In [160]:
from sklearn.linear_model import Lasso

In [161]:
logit = Lasso()

In [162]:
logit.fit(X_train, y_train)

Lasso()

In [163]:
LassoDF = pd.DataFrame(logit.coef_)
G = pd.concat([pd.Series(X_train.columns), LassoDF], axis = 1)
G.columns = ['Feature', 'Lasso Coefficient']
G.sort_values('Lasso Coefficient')

,Feature,Lasso Coefficient
82,Condition2_PosN,-418075.456269
153,Functional_Mod,-33455.951199
105,Exterior1st_CemntBd,-31238.180389
47,LandSlope_Sev,-30501.139813
122,Exterior2nd_Stone,-21097.409765
...,...,...
172,MiscFeature_Shed,56120.945598
30,MSZoning_RH,56777.267896
29,MSZoning_FV,70529.230967
171,MiscFeature_Othr,96909.189096


In [164]:
logit.score(X_test, y_test)

0.7515899865193028

In [165]:
LassoPred = logit.predict(X_test)

In [166]:
LassoPred

array([190461.56947681, 169953.29210824, 269923.01366915, 149031.96625407,
       209519.40095341, 110385.07844048, 131094.85985548, 403251.22180802,
       207466.34228476, 124480.0812373 , 145823.56852559, 119030.2982422 ,
       198500.46399071, 122426.75038143, 322820.47884814, 106431.84099799,
       248283.31360935, 173809.31969593,  81812.5564477 ,  84873.57515765,
       334678.60142489, 259711.7126535 , 268321.6880555 , 135659.27204404,
       197457.12269405, 147412.45909593, 201191.71024273, 137989.89164789,
       196754.59272955, 115599.50580691, 179069.02895249,  80080.27634883,
       139570.52930143, 133864.29467868,  39144.70349996, 137945.42902896,
       180872.21131465, 198696.91078433, 139716.76653675, 242300.44745446,
       241823.96110314, 205593.70694032, 614286.55319562, 192635.21685692,
        84541.32690157, 255003.15927006, 103722.26489884, 171192.37077016,
       157432.44941646, 168964.50163732, 119230.22286691, 140670.45721167,
       215560.54507597, 1

## How about a Random Forest

In [167]:
from sklearn.ensemble import RandomForestRegressor

In [168]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
rfr.score(X_test, y_test)

0.8940869076840345

### Let's do some parameter tuning

In [169]:
params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [12, 14, 16],
    'min_samples_leaf': [4, 5],
    'min_samples_split': [4, 5],
    'max_leaf_nodes': [500, 600, 700],
    'max_samples': [None],
    
}
grid = GridSearchCV(rfr, params, refit=True, verbose=3, n_jobs=-1)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


0.8950066085538795

In [170]:
grid.best_params_

{'max_depth': 12,
 'max_leaf_nodes': 500,
 'max_samples': None,
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 300}

In [171]:
rfrPred = pd.Series(rfr.predict(X_test))
rfrPred

0      165466.15
1      170866.24
2      254758.59
3      166066.24
4      208871.20
         ...    
428    113334.50
429    205053.85
430    160697.37
431    129476.62
432     95671.47
Length: 433, dtype: float64

## XG Boost?

In [197]:
import xgboost as xgb

In [198]:
logitXB = xgb.XGBRegressor()
logitXB.fit(X_train, y_train)
logitXB.score(X_test, y_test)

0.8595204823702188

In [199]:
from sklearn.metrics import mean_absolute_error

In [200]:
mean_train = np.mean(y_train)
mean_train[0]
mean_test = np.mean(y_test)

In [201]:
baseline_predictions = np.ones(y_test.shape) * mean_train[0]
baseline_predictions

array([[181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [181590.47175421],
       [1815

In [202]:
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print(f'Baseline MAE is {round(mae_baseline,2)}')

Baseline MAE is 59188.76


In [203]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [204]:
params['eval_metric'] = "mae"

In [205]:
num_boost_round = 999

In [206]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [207]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[17:13:45] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:126362.89844
[1]	Test-mae:88959.07031
[2]	Test-mae:63119.57812
[3]	Test-mae:45620.48828
[4]	Test-mae:34316.16406
[5]	Test-mae:26778.93945
[6]	Test-mae:22807.87305
[7]	Test-mae:20783.06836
[8]	Test-mae:19691.68359
[9]	Test-mae:19182.65039
[10]	Test-mae:18806.04688
[11]	Test-mae:18639.15430
[12]	Test-mae:18437.89062
[13]	Test-mae:18361.50000
[14]	Test-mae:18306.71094
[15]	Test-mae:18290.52930
[16]	Test-mae:18251.86133
[17]	Test-mae:18277.38867
[18]	Test-mae:18244.77539
[19]	Test-mae:18152.32812
[20]	Test-mae:18115.19336
[21]	Test-mae:18105.68164
[22]	Test-mae:18102.78711
[23]	Test-mae:18089.91992
[24]	Test-mae:18014.96484
[25]	Test-mae:17934.12891
[26]	Test-mae:17921.73438
[27]	Test-mae:17910.60742
[28]	Test-mae:17899.75391
[29]	Test-mae:17921.63672
[30]	Test-mae:17950.76172
[31]	Test-mae:17975.36523
[32]	Test-mae:17981.34

In [208]:
print(f'Best MAE is {round(model.best_score, 2)} with {round(model.best_iteration+1, 2)} rounds')

Best MAE is 17899.75 with 29 rounds


In [209]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

[17:14:00] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:00] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:00] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:00] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:00] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,127991.901563,1884.335019,128195.006250,8759.942644
1,90413.001563,1380.432031,91002.339063,7352.987668
2,64099.654688,1050.527348,65074.317969,6120.519523
3,45607.996094,751.748147,47546.600782,5289.402650
4,32696.480860,570.781060,36215.062109,4749.841818
5,23921.833984,442.874715,29502.402344,4137.707530
6,18140.742969,331.632932,25530.606250,3593.038291
7,14494.142383,239.895656,23202.145703,3080.474261
8,12200.346485,185.121441,22033.956250,2936.115829
9,10657.714648,95.776295,21442.521875,2823.744598


In [210]:
cv_results['test-mae-mean'].min()

20089.3203124

In [211]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(8,13)
    for min_child_weight in range(5,10)
]

In [212]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=8, min_child_weight=5
[17:14:10] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:10] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:10] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:10] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:10] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 19783.750781400002 for 22 rounds
CV with max_depth=8, min_child_weight=6
[17:14:11] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear 

	MAE 19291.1007812 for 18 rounds
CV with max_depth=10, min_child_weight=5
[17:14:29] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:29] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:29] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:29] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:29] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 19820.9828126 for 36 rounds
CV with max_depth=10, min_child_weight=6
[17:14:32] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regr

	MAE 19338.5052738 for 20 rounds
CV with max_depth=12, min_child_weight=5
[17:14:46] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:46] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:46] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:46] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:46] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 19818.5591798 for 19 rounds
CV with max_depth=12, min_child_weight=6
[17:14:48] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regr

In [119]:
params['max_depth'] = 8
params['min_child_weight'] = 9

In [120]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [121]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
[16:47:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:35] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 19140.4281252 for 24 rounds
CV with subsample=1.0, colsample=0.9
[16:47:37] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now depr

	MAE 19332.6011716 for 30 rounds
CV with subsample=0.8, colsample=0.8
[16:47:51] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:51] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:51] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:51] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:47:51] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
	MAE 20117.863672 for 36 rounds
CV with subsample=0.8, colsample=0.7
[16:47:52] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_ob

In [213]:
params['subsample'] = 1
params['colsample_bytree'] = 0.8

In [214]:
%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time cv_results = xgb.cv(params, dtrain,num_boost_round=num_boost_round,seed=42,nfold=5, metrics=['mae'], early_stopping_rounds=10)    
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

Wall time: 0 ns
CV with eta=0.3
[17:14:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[17:14:55] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
Wall time: 1.43 s
	MAE 19112.866797000002 for 14 rounds

CV with eta=0.2
[17:14:56] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprec

In [217]:
params['eta'] = .01

In [218]:
num_boost_round

999

In [219]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[17:17:39] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:178218.90625
[1]	Test-mae:176435.53125
[2]	Test-mae:174669.76562
[3]	Test-mae:172925.46875
[4]	Test-mae:171199.56250
[5]	Test-mae:169483.60938
[6]	Test-mae:167792.01562
[7]	Test-mae:166128.73438
[8]	Test-mae:164471.25000
[9]	Test-mae:162826.10938
[10]	Test-mae:161198.67188
[11]	Test-mae:159583.70312
[12]	Test-mae:157984.46875
[13]	Test-mae:156404.68750
[14]	Test-mae:154844.89062
[15]	Test-mae:153303.76562
[16]	Test-mae:151782.87500
[17]	Test-mae:150262.79688
[18]	Test-mae:148769.59375
[19]	Test-mae:147286.34375
[20]	Test-mae:145811.48438
[21]	Test-mae:144354.21875
[22]	Test-mae:142910.87500
[23]	Test-mae:141483.62500
[24]	Test-mae:140072.78125
[25]	Test-mae:138681.93750
[26]	Test-mae:137303.35938
[27]	Test-mae:135937.15625
[28]	Test-mae:134587.82812
[29]	Test-mae:133241.12500
[30]	Test-mae:131909.53125
[31]	Test-mae:1306

[300]	Test-mae:18450.71094
[301]	Test-mae:18413.01758
[302]	Test-mae:18383.24805
[303]	Test-mae:18351.56836
[304]	Test-mae:18322.95312
[305]	Test-mae:18296.44531
[306]	Test-mae:18272.27734
[307]	Test-mae:18239.93945
[308]	Test-mae:18211.01562
[309]	Test-mae:18187.29102
[310]	Test-mae:18159.41797
[311]	Test-mae:18132.41992
[312]	Test-mae:18107.59766
[313]	Test-mae:18077.94727
[314]	Test-mae:18054.76172
[315]	Test-mae:18032.16797
[316]	Test-mae:18003.78125
[317]	Test-mae:17975.83398
[318]	Test-mae:17952.33008
[319]	Test-mae:17928.87305
[320]	Test-mae:17901.48047
[321]	Test-mae:17876.61719
[322]	Test-mae:17854.39258
[323]	Test-mae:17830.54688
[324]	Test-mae:17809.83008
[325]	Test-mae:17783.48828
[326]	Test-mae:17757.66016
[327]	Test-mae:17737.21094
[328]	Test-mae:17720.84766
[329]	Test-mae:17698.46680
[330]	Test-mae:17674.68164
[331]	Test-mae:17653.01172
[332]	Test-mae:17629.91016
[333]	Test-mae:17614.19727
[334]	Test-mae:17594.84961
[335]	Test-mae:17579.78906
[336]	Test-mae:17559.23242
[

In [220]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[17:17:57] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:178218.90625
[1]	Test-mae:176435.53125
[2]	Test-mae:174669.76562
[3]	Test-mae:172925.46875
[4]	Test-mae:171199.56250
[5]	Test-mae:169483.60938
[6]	Test-mae:167792.01562
[7]	Test-mae:166128.73438
[8]	Test-mae:164471.25000
[9]	Test-mae:162826.10938
[10]	Test-mae:161198.67188
[11]	Test-mae:159583.70312
[12]	Test-mae:157984.46875
[13]	Test-mae:156404.68750
[14]	Test-mae:154844.89062
[15]	Test-mae:153303.76562
[16]	Test-mae:151782.87500
[17]	Test-mae:150262.79688
[18]	Test-mae:148769.59375
[19]	Test-mae:147286.34375
[20]	Test-mae:145811.48438
[21]	Test-mae:144354.21875
[22]	Test-mae:142910.87500
[23]	Test-mae:141483.62500
[24]	Test-mae:140072.78125
[25]	Test-mae:138681.93750
[26]	Test-mae:137303.35938
[27]	Test-mae:135937.15625
[28]	Test-mae:134587.82812
[29]	Test-mae:133241.12500
[30]	Test-mae:131909.53125
[31]	Test-mae:1306

[300]	Test-mae:18450.71094
[301]	Test-mae:18413.01758
[302]	Test-mae:18383.24805
[303]	Test-mae:18351.56836
[304]	Test-mae:18322.95312
[305]	Test-mae:18296.44531
[306]	Test-mae:18272.27734
[307]	Test-mae:18239.93945
[308]	Test-mae:18211.01562
[309]	Test-mae:18187.29102
[310]	Test-mae:18159.41797
[311]	Test-mae:18132.41992
[312]	Test-mae:18107.59766
[313]	Test-mae:18077.94727
[314]	Test-mae:18054.76172
[315]	Test-mae:18032.16797
[316]	Test-mae:18003.78125
[317]	Test-mae:17975.83398
[318]	Test-mae:17952.33008
[319]	Test-mae:17928.87305
[320]	Test-mae:17901.48047
[321]	Test-mae:17876.61719
[322]	Test-mae:17854.39258
[323]	Test-mae:17830.54688
[324]	Test-mae:17809.83008
[325]	Test-mae:17783.48828
[326]	Test-mae:17757.66016
[327]	Test-mae:17737.21094
[328]	Test-mae:17720.84766
[329]	Test-mae:17698.46680
[330]	Test-mae:17674.68164
[331]	Test-mae:17653.01172
[332]	Test-mae:17629.91016
[333]	Test-mae:17614.19727
[334]	Test-mae:17594.84961
[335]	Test-mae:17579.78906
[336]	Test-mae:17559.23242
[

In [221]:
mean_absolute_error(best_model.predict(dtest), y_test)

16852.15213625866

In [222]:
params

{'max_depth': 12,
 'min_child_weight': 9,
 'eta': 0.01,
 'subsample': 1,
 'colsample_bytree': 0.8,
 'objective': 'reg:linear',
 'eval_metric': 'mae'}

In [254]:
best_model.save_model("my_model.model")

In [255]:
loaded_model = xgb.Booster()
loaded_model.load_model("my_model.model")
loaded_model.predict(dtest)

[08:14:03] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


array([163533.94 , 172923.88 , 250018.9  , 153238.88 , 176187.88 ,
       132577.08 , 138795.53 , 418638.7  , 193508.55 , 119934.14 ,
       143900.73 , 137495.31 , 173661.8  , 122769.62 , 291082.7  ,
       125589.37 , 289534.56 , 184986.2  , 104849.336, 105963.1  ,
       337818.72 , 273570.75 , 257961.69 , 125939.11 , 198970.95 ,
       162032.27 , 198011.73 , 146060.25 , 169940.   , 112002.945,
       168100.86 , 112548.24 , 137865.08 , 136107.64 ,  98517.78 ,
       111493.58 , 159207.94 , 201768.61 , 155665.64 , 227126.39 ,
       252942.05 , 190037.78 , 350645.53 , 169838.22 ,  91461.49 ,
       222781.52 , 116237.2  , 229281.31 , 158694.86 , 164509.61 ,
       123221.01 , 152498.   , 173932.94 , 108691.19 , 237523.39 ,
       147496.4  , 123139.195, 223771.55 , 130331.82 ,  75780.98 ,
        89424.22 , 104218.664, 177789.66 , 173089.8  , 188701.86 ,
       112082.94 , 244134.16 , 177167.69 , 408321.4  , 206995.69 ,
       405748.16 , 216272.62 , 173434.06 , 205796.98 , 345732.

In [225]:
RSquared = 1 - (mean_absolute_error(best_model.predict(dtest), y_test)/mean_test[0])

In [226]:
RSquared

0.9063820434873616

# Let's Put all our Models together for some Ensembling 

In [227]:
FullPredicitions = pd.concat([pd.Series(loaded_model.predict(dtest)), y_test.SalePrice], axis = 1)
FullPredicitions.columns = ['XGBoost', 'Actual']

In [228]:
FullPredictions = pd.concat([pd.Series(LassoPred), FullPredicitions], axis = 1)
FullPredictions.columns = ['Lasso','XGBoost', 'Actual']
FullPredictions

,Lasso,XGBoost,Actual
0,190461.569477,163533.937500,173000
1,169953.292108,172923.875000,135000
2,269923.013669,250018.906250,267000
3,149031.966254,153238.875000,161000
4,209519.400953,176187.875000,136000
...,...,...,...
428,107438.498975,110942.367188,135750
429,185210.247674,198652.671875,192500
430,150924.239416,160164.156250,146500
431,110909.123278,130279.757812,118500


In [229]:
FullPredictions = pd.concat([PredGrid.Predicted, FullPredictions], axis = 1)
FullPredictions.columns = ['ElasticNet','Lasso','XGBoost', 'Actual']
FullPredictions

,ElasticNet,Lasso,XGBoost,Actual
0,194028.747739,190461.569477,163533.937500,173000
1,180063.554362,169953.292108,172923.875000,135000
2,262257.165281,269923.013669,250018.906250,267000
3,149214.195912,149031.966254,153238.875000,161000
4,193498.706182,209519.400953,176187.875000,136000
...,...,...,...,...
428,109061.925443,107438.498975,110942.367188,135750
429,196448.084812,185210.247674,198652.671875,192500
430,157902.336025,150924.239416,160164.156250,146500
431,114772.204684,110909.123278,130279.757812,118500


In [230]:
FullPredictions = pd.concat([rfrPred, FullPredictions], axis = 1)
FullPredictions.columns = ['RandomForest','ElasticNet','Lasso','XGBoost', 'Actual']
FullPredictions

,RandomForest,ElasticNet,Lasso,XGBoost,Actual
0,165466.15,194028.747739,190461.569477,163533.937500,173000
1,170866.24,180063.554362,169953.292108,172923.875000,135000
2,254758.59,262257.165281,269923.013669,250018.906250,267000
3,166066.24,149214.195912,149031.966254,153238.875000,161000
4,208871.20,193498.706182,209519.400953,176187.875000,136000
...,...,...,...,...,...
428,113334.50,109061.925443,107438.498975,110942.367188,135750
429,205053.85,196448.084812,185210.247674,198652.671875,192500
430,160697.37,157902.336025,150924.239416,160164.156250,146500
431,129476.62,114772.204684,110909.123278,130279.757812,118500


In [231]:
meltTemp = pd.melt(FullPredictions, id_vars='Actual')
meltTemp
#fig = px.scatter(meltTemp.loc[meltTemp['variable'].isin(['RandomForest', 'Lasso', 'ElasticNet','XGBoost'])], x = 'Actual', y = 'value', color = 'variable')
#fig


,Actual,variable,value
0,173000,RandomForest,165466.150000
1,135000,RandomForest,170866.240000
2,267000,RandomForest,254758.590000
3,161000,RandomForest,166066.240000
4,136000,RandomForest,208871.200000
...,...,...,...
1727,135750,XGBoost,110942.367188
1728,192500,XGBoost,198652.671875
1729,146500,XGBoost,160164.156250
1730,118500,XGBoost,130279.757812


In [232]:
FullPredictions['RandomForest_Variance'] = np.sqrt((FullPredictions['RandomForest'] - FullPredictions['Actual'])**2)
FullPredictions['ElasticNet_Variance'] = np.sqrt((FullPredictions['ElasticNet'] - FullPredictions['Actual'])**2)
FullPredictions['Lasso_Variance'] = np.sqrt((FullPredictions['Lasso'] - FullPredictions['Actual'])**2)
FullPredictions['XGBoost_Variance'] = np.sqrt((FullPredictions['XGBoost'] - FullPredictions['Actual'])**2)
FullPredictions

,RandomForest,ElasticNet,Lasso,XGBoost,Actual,RandomForest_Variance,ElasticNet_Variance,Lasso_Variance,XGBoost_Variance
0,165466.15,194028.747739,190461.569477,163533.937500,173000,7533.85,21028.747739,17461.569477,9466.062500
1,170866.24,180063.554362,169953.292108,172923.875000,135000,35866.24,45063.554362,34953.292108,37923.875000
2,254758.59,262257.165281,269923.013669,250018.906250,267000,12241.41,4742.834719,2923.013669,16981.093750
3,166066.24,149214.195912,149031.966254,153238.875000,161000,5066.24,11785.804088,11968.033746,7761.125000
4,208871.20,193498.706182,209519.400953,176187.875000,136000,72871.20,57498.706182,73519.400953,40187.875000
...,...,...,...,...,...,...,...,...,...
428,113334.50,109061.925443,107438.498975,110942.367188,135750,22415.50,26688.074557,28311.501025,24807.632812
429,205053.85,196448.084812,185210.247674,198652.671875,192500,12553.85,3948.084812,7289.752326,6152.671875
430,160697.37,157902.336025,150924.239416,160164.156250,146500,14197.37,11402.336025,4424.239416,13664.156250
431,129476.62,114772.204684,110909.123278,130279.757812,118500,10976.62,3727.795316,7590.876722,11779.757812


In [233]:
ScoreDF = pd.DataFrame({
    'RandomForestMAE': [FullPredictions.RandomForest_Variance.sum()/(len(FullPredictions.index))],
    'ElasticNetMAE': [FullPredictions.ElasticNet_Variance.sum()/(len(FullPredictions.index))],
    'LassoMAE': [FullPredictions.Lasso_Variance.sum()/(len(FullPredictions.index))],
    'XGBoostMAE': [FullPredictions.XGBoost_Variance.sum()/(len(FullPredictions.index))]
})
ScoreDF

    

,RandomForestMAE,ElasticNetMAE,LassoMAE,XGBoostMAE
0,17752.23157,20421.826648,22069.887807,16852.152136


In [234]:
ScoreDF['RandomForestRSquared'] = 1-(ScoreDF.RandomForestMAE/mean_test[0])
ScoreDF['ElasticNetRSquared'] = 1-(ScoreDF.ElasticNetMAE/mean_test[0])
ScoreDF['LassoRSquared'] = 1-(ScoreDF.LassoMAE/mean_test[0])
ScoreDF['XGBoostRSquared'] = 1-(ScoreDF.XGBoostMAE/mean_test[0])



In [235]:
ScoreDF

,RandomForestMAE,ElasticNetMAE,LassoMAE,XGBoostMAE,RandomForestRSquared,ElasticNetRSquared,LassoRSquared,XGBoostRSquared
0,17752.23157,20421.826648,22069.887807,16852.152136,0.901382,0.886552,0.877396,0.906382


In [236]:
EnsembleDF = FullPredictions.loc[:,['RandomForest', 'Lasso', 'XGBoost', 'ElasticNet', 'Actual']]

loopDF = FullPredictions.loc[:,['Actual']]
minMAE = float('Inf')
pLst = []
for i in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
    for j in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
        for k in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
            for m in [1, .9, .8, .7, .6, .5, .4, .3, .2, .1, 0]:
                if (i + j + k + m) == 1:
                    loopDF['RandomForest'] = FullPredictions.RandomForest * i
                    loopDF['Lasso'] = FullPredictions.Lasso * j
                    loopDF['XGBoost'] = FullPredictions.XGBoost * k
                    loopDF['ElasticNet'] = FullPredictions.ElasticNet * m
                    loopDF['Prediction'] = loopDF['ElasticNet'] + loopDF['XGBoost'] + loopDF['Lasso'] + loopDF['RandomForest']
                    loopDF['Difference'] = loopDF['Prediction'] - loopDF['Actual']
                    loopDF['Diffsq'] = np.sqrt(loopDF['Difference']**2)
                    mae = loopDF.Diffsq.sum()/len(loopDF.index)
                    if mae < minMAE:
                        minMAE = mae
                        pLst = [i,j,k,m]
                        print(mae)


                        
        
    
    

17752.2315704388
17077.901968524962
16745.522050677082
16650.142412191133
16400.239699576756
16295.935196687258
16221.1292459803
16171.239440389421
16156.261022402383


In [237]:
pLst

[0.2, 0.2, 0.6, 0]

In [238]:
minMAE

16156.261022402383

# Ensembling Complete:
### RandomForest = 0.0
### Lasso = 0.2
### XGBoost = 0.8

RandomState 0 gives MAE of 15,147m

RandomState 1 gives MAE of 15,836

# Let's Predict

In [257]:
# Reorder Columns in finalDF to match X_train
finalDF = finalDF[X_train.columns]

In [279]:
# Lasso Predicitons
finLasPred = logit.predict(finalDF)

In [280]:
# RandomForest Predictions
finRFRPred = grid.predict(finalDF)

In [275]:
# Create labels DF for XGBoost
df_y = pd.DataFrame(np.zeros((1448,1)))
df_y.columns = ['SalePrice']
df_y

,SalePrice
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1443,0.0
1444,0.0
1445,0.0
1446,0.0


In [281]:
# XGBoost Prediction
finXGBPred = loaded_model.predict(xgb.DMatrix(finalDF, label=df_y))

## Bring all Predictions together and analyize

In [292]:
FinalPred = pd.concat([pd.Series(finLasPred), pd.Series(finRFRPred), pd.Series(finXGBPred)], axis = 1)
FinalPred.columns = ['Lasso_Predictions', 'RandomForest_Predictions', 'XGBoost_Predictions']
FinalPred

,Lasso_Predictions,RandomForest_Predictions,XGBoost_Predictions
0,105226.862453,124663.210725,119275.328125
1,185627.607864,155296.295728,155742.640625
2,201500.329708,191500.687088,191272.234375
3,193428.741078,190758.029909,192654.343750
4,206726.687896,189464.508565,189544.140625
...,...,...,...
1443,72469.797253,94792.448891,89462.234375
1444,76227.451070,97361.239747,92041.257812
1445,176361.557348,179658.817505,174870.375000
1446,120681.195671,110737.334779,117449.234375


## Let's Visualize the Deltas of all our predictions against eachother

In [291]:
AA = FinalPred
AA['Delta:Lasso-RanForest'] = AA.RandomForest_Predictions - AA.Lasso_Predictions
AA['Delta:Lasso-XGBoost'] = AA.XGBoost_Predictions - AA.Lasso_Predictions
AA['Delta:XGBoost-RanForest'] = AA.RandomForest_Predictions - AA.XGBoost_Predictions
AA.drop(['Lasso_Predictions', 'RandomForest_Predictions', 'XGBoost_Predictions'], axis =1, inplace = True)
AAmelt = pd.melt(AA)
AAmelt



,variable,value
0,Delta:Lasso-RanForest,19436.348272
1,Delta:Lasso-RanForest,-30331.312136
2,Delta:Lasso-RanForest,-9999.642621
3,Delta:Lasso-RanForest,-2670.711169
4,Delta:Lasso-RanForest,-17262.179331
...,...,...
4339,Delta:XGBoost-RanForest,5330.214516
4340,Delta:XGBoost-RanForest,5319.981935
4341,Delta:XGBoost-RanForest,4788.442505
4342,Delta:XGBoost-RanForest,-6711.899596


In [311]:
fig = px.box(AAmelt, y = 'value', x = 'variable', color_discrete_sequence = ['#454b59'], title = 'Difference Between 3 Predictive Models')
fig.update_layout(
    xaxis_title = 'Model',
    yaxis_title = 'Delta Sale Price ($)',
    title_x=0.5,
    plot_bgcolor = '#d0c5b5',
    title_font = dict(size = 25),
    yaxis = dict(
        tickfont = dict(size=14),
        titlefont = dict(size = 25),
        showline = True,
        linecolor = 'black',
        range = [-150000,150000]
    ),
    xaxis = dict(
        tickfont = dict(size=16),
        titlefont = dict(size =25),

        linecolor = 'black'        
    )    
)
for t in fig.data:
    t.marker.line.width = 1
    t.marker.line.color = "black"
fig

## Random Forest and XGBoost agree with each other the most
**Will use 80% XGBoost and 20% RandomForest**

In [312]:
FinalPredictions = (0.8 * FinalPred['XGBoost_Predictions']) + (.2* FinalPred['RandomForest_Predictions'])

## Let's look at the distribution between my predicitions and the given dataset

In [318]:
BB = pd.DataFrame(FinalPredictions)
BB['TrainSalePrice'] = y_train['SalePrice']
BB.columns = ['Predicted Sale Price', 'Disclosed Sale Price']
BBmelt = pd.melt(BB)
BBmelt

,variable,value
0,Predicted Sale Price,120352.90777
1,Predicted Sale Price,155653.376333
2,Predicted Sale Price,191317.934293
3,Predicted Sale Price,192275.090357
4,Predicted Sale Price,189528.214213
...,...,...
2893,Disclosed Sale Price,NaN
2894,Disclosed Sale Price,NaN
2895,Disclosed Sale Price,NaN
2896,Disclosed Sale Price,NaN


In [320]:
fig = px.box(BBmelt, y = 'value', x = 'variable', color_discrete_sequence = ['#454b59'], title = 'Distribution of Predicted and Disclosed Sale Prices')
fig.update_layout(
    xaxis_title = 'Type',
    yaxis_title = 'Sale Price ($)',
    title_x=0.5,
    plot_bgcolor = '#d0c5b5',
    title_font = dict(size = 25),
    yaxis = dict(
        tickfont = dict(size=14),
        titlefont = dict(size = 25),
        showline = True,
        linecolor = 'black'
       
    ),
    xaxis = dict(
        tickfont = dict(size=16),
        titlefont = dict(size =25),

        linecolor = 'black'        
    )    
)
for t in fig.data:
    t.marker.line.width = 1
    t.marker.line.color = "black"
fig

**distributions look pretty similar and are centered pretty close to each other**